In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_excel('DataSources/CC-3Y-Transactions.xlsx')
data.columns = data.columns.str.lstrip()
data.tail(10)
fdata = data
fdata

,City,Date,Card Type,Exp Type,Gender,Amount
0,"Delhi, India",2014-10-29,Gold,Bills,F,82475
1,"Greater Mumbai, India",2014-08-22,Platinum,Bills,F,32555
2,"Bengaluru, India",2014-08-27,Silver,Bills,F,101738
3,"Greater Mumbai, India",2014-04-12,Signature,Bills,F,123424
4,"Bengaluru, India",2015-05-05,Gold,Bills,F,171574
...,...,...,...,...,...,...
26047,"Kolkata, India",2014-06-22,Silver,Travel,F,128191
26048,"Pune, India",2014-08-03,Signature,Travel,M,246316
26049,"Hyderabad, India",2015-01-16,Silver,Travel,M,265019
26050,"Kanpur, India",2014-09-14,Silver,Travel,M,88174


# Rules:
Assumption - Card Luxury Rank (smaller rank is better card) - 1.Platinum, 2.Gold, 3.Silver, 4.Signature

Business Rules: 
Rule-1 - Limits:
Signature Limit - 250,000.
Silver Limit - 450,000.
Gold Limit - 650,000.
Platinum Limit - 950,000.

Rule-2 - Valid Combined Exp Type (example, bills can be combined of multiple things such as power bill, rent, and so on. same for entertainment and travel)
Transaction amount cannot be more than the 75th percentile of total amount spent on certain expense type.
Bills - 237895
Entertainment - 228414
Travel - 221778

Rule-3 - Valid Individual Exp Type
Transaction amount cannot be more than the 25th percentile of total amount spent on certain expense type.
Food - 75862
Fuel - 77405
Grocery - 75648

Rule-4 - Gender Expense Check
A transaction amount cannot be more than the 75th percentile of total amount spent by certain gender.
Male:  225900.0
Female:  230407.75

Rule-5 - Metro cities check
Top 5 metro cities cannot have expense amount more than the 75th percentile of total amount spent.
Delhi:  231531.75
Greater Mumbai:  233161.0
Pune:  224771.5
Hyderabad:  212849.25
Kolkata:  216616.0

Rule-6 - Minor cities check
All cities except metro cities cannot have expense amount more than the 50th percentile of average amount spent.
Non-Metro Cities:  153289.0

# Rule 1,2,3

In [3]:
#Rule-1
fdata.loc[(fdata['Amount']>250000) & (fdata['Card Type']=='Signature'), 'Limit_Check']=1
fdata.loc[(fdata['Amount']>450000) & (fdata['Card Type']=='Silver'), 'Limit_Check']=1
fdata.loc[(fdata['Amount']>650000) & (fdata['Card Type']=='Gold'), 'Limit_Check']=1
fdata.loc[(fdata['Amount']>950000) & (fdata['Card Type']=='Platinum'), 'Limit_Check']=1

rule1 = fdata.loc[fdata['Limit_Check']==1]
rule1 = rule1[["City", "Date", "Card Type", "Exp Type", "Gender", "Amount"]]
print('Data Exlpoiting Rule 1')
rule1.reset_index(drop=True)

Data Exlpoiting Rule 1


,City,Date,Card Type,Exp Type,Gender,Amount
0,"Greater Mumbai, India",2013-12-07,Signature,Bills,F,278036
1,"Delhi, India",2014-04-27,Signature,Bills,F,254359
2,"Greater Mumbai, India",2014-08-15,Signature,Bills,F,302834
3,"Greater Mumbai, India",2014-06-14,Signature,Bills,F,421878
4,"Greater Mumbai, India",2015-03-30,Gold,Bills,F,986379
...,...,...,...,...,...,...
1251,"Chennai, India",2014-10-25,Signature,Travel,M,272123
1252,"Chennai, India",2014-04-03,Signature,Travel,M,270128
1253,"Hyderabad, India",2015-02-11,Signature,Travel,F,254461
1254,"Kanpur, India",2014-07-07,Signature,Travel,M,262233


In [4]:
#Rule-2
print('75th Percentile of Amount Spent on Bills: ', fdata.loc[fdata['Exp Type']=='Bills','Amount'].quantile(0.75))
print('75th Percentile of Amount Spent on Entertainment: ', fdata.loc[fdata['Exp Type']=='Entertainment','Amount'].quantile(0.75))
print('75th Percentile of Amount Spent on Travel: ', fdata.loc[fdata['Exp Type']=='Travel','Amount'].quantile(0.75))
fdata.loc[(fdata['Amount']>237895) & (fdata['Exp Type']=='Bills'), 'ExpenseType_Check']=1
fdata.loc[(fdata['Amount']>228414) & (fdata['Exp Type']=='Entertainment'), 'ExpenseType_Check']=1
fdata.loc[(fdata['Amount']>221778) & (fdata['Exp Type']=='Travel'), 'ExpenseType_Check']=1

rule2 = fdata.loc[fdata['ExpenseType_Check']==1]
rule2 = rule2[["City", "Date", "Card Type", "Exp Type", "Gender", "Amount"]]
print('Data Exlpoiting Rule 2')
rule2.reset_index(drop=True)

75th Percentile of Amount Spent on Bills:  237895.5
75th Percentile of Amount Spent on Entertainment:  228414.75
75th Percentile of Amount Spent on Travel:  221778.75
Data Exlpoiting Rule 2


,City,Date,Card Type,Exp Type,Gender,Amount
0,"Delhi, India",2014-06-22,Platinum,Bills,F,280061
1,"Greater Mumbai, India",2013-12-07,Signature,Bills,F,278036
2,"Delhi, India",2014-04-27,Signature,Bills,F,254359
3,"Greater Mumbai, India",2014-08-15,Signature,Bills,F,302834
4,"Greater Mumbai, India",2014-11-28,Platinum,Bills,F,647116
...,...,...,...,...,...,...
2641,"Surat, India",2015-05-19,Gold,Travel,M,269464
2642,"Hyderabad, India",2015-05-15,Silver,Travel,F,280335
2643,"Kolkata, India",2014-10-14,Gold,Travel,M,243249
2644,"Pune, India",2014-08-03,Signature,Travel,M,246316


In [5]:
#Rule-3
print('25th Percentile of Amount Spent on Food: ', fdata.loc[fdata['Exp Type']=='Food','Amount'].quantile(0.25))
print('25th Percentile of Amount Spent on Fuel: ', fdata.loc[fdata['Exp Type']=='Fuel','Amount'].quantile(0.25))
print('25th Percentile of Amount Spent on Grocery: ', fdata.loc[fdata['Exp Type']=='Grocery','Amount'].quantile(0.25))
fdata.loc[(fdata['Amount']>75862) & (fdata['Exp Type']=='Food'), 'ExpenseType_Check']=1
fdata.loc[(fdata['Amount']>77405) & (fdata['Exp Type']=='Fuel'), 'ExpenseType_Check']=1
fdata.loc[(fdata['Amount']>75648) & (fdata['Exp Type']=='Grocery'), 'ExpenseType_Check']=1

rule3 = fdata.loc[fdata['ExpenseType_Check']==1]
rule3 = rule3[["City", "Date", "Card Type", "Exp Type", "Gender", "Amount"]]
print('Data Exlpoiting Rule 3')
rule3.reset_index(drop=True)

25th Percentile of Amount Spent on Food:  75862.5
25th Percentile of Amount Spent on Fuel:  77405.0
25th Percentile of Amount Spent on Grocery:  75648.25
Data Exlpoiting Rule 3


,City,Date,Card Type,Exp Type,Gender,Amount
0,"Delhi, India",2014-06-22,Platinum,Bills,F,280061
1,"Greater Mumbai, India",2013-12-07,Signature,Bills,F,278036
2,"Delhi, India",2014-04-27,Signature,Bills,F,254359
3,"Greater Mumbai, India",2014-08-15,Signature,Bills,F,302834
4,"Greater Mumbai, India",2014-11-28,Platinum,Bills,F,647116
...,...,...,...,...,...,...
14245,"Surat, India",2015-05-19,Gold,Travel,M,269464
14246,"Hyderabad, India",2015-05-15,Silver,Travel,F,280335
14247,"Kolkata, India",2014-10-14,Gold,Travel,M,243249
14248,"Pune, India",2014-08-03,Signature,Travel,M,246316


In [6]:
#Rule-4
print('75th Percentile of Amount Spent by Male: ', fdata.loc[fdata['Gender']=='M','Amount'].quantile(0.75))
print('75th Percentile of Amount Spent by Female: ', fdata.loc[fdata['Gender']=='F','Amount'].quantile(0.75))
fdata.loc[(fdata['Amount']>225900) & (fdata['Gender']=='M'), 'GenExp_Check']=1
fdata.loc[(fdata['Amount']>230407) & (fdata['Gender']=='F'), 'GenExp_Check']=1

rule4 = fdata.loc[fdata['GenExp_Check']==1]
rule4 = rule4[["City", "Date", "Card Type", "Exp Type", "Gender", "Amount"]]
print('Data Exlpoiting Rule 3')
rule4.reset_index(drop=True)

75th Percentile of Amount Spent by Male:  225900.0
75th Percentile of Amount Spent by Female:  230407.75
Data Exlpoiting Rule 3


,City,Date,Card Type,Exp Type,Gender,Amount
0,"Delhi, India",2014-06-22,Platinum,Bills,F,280061
1,"Greater Mumbai, India",2013-12-07,Signature,Bills,F,278036
2,"Delhi, India",2014-04-27,Signature,Bills,F,254359
3,"Greater Mumbai, India",2014-08-15,Signature,Bills,F,302834
4,"Greater Mumbai, India",2014-11-28,Platinum,Bills,F,647116
...,...,...,...,...,...,...
6508,"Surat, India",2015-05-19,Gold,Travel,M,269464
6509,"Hyderabad, India",2015-05-15,Silver,Travel,F,280335
6510,"Kolkata, India",2014-10-14,Gold,Travel,M,243249
6511,"Pune, India",2014-08-03,Signature,Travel,M,246316


In [7]:
#Rule-5
#top 5 metro cities = ['Delhi, India', 'Greater Mumbai, India', 'Pune, India', 'Hyderabad, India', 'Kolkata, India']
print('75th Percentile of Amount Spent in Delhi: ', fdata.loc[fdata['City']=='Delhi, India','Amount'].quantile(0.75))
print('75th Percentile of Amount Spent by Greater Mumbai: ', fdata.loc[fdata['City']=='Greater Mumbai, India','Amount'].quantile(0.75))
print('75th Percentile of Amount Spent by Pune: ', fdata.loc[fdata['City']=='Pune, India','Amount'].quantile(0.75))
print('75th Percentile of Amount Spent by Hyderabad: ', fdata.loc[fdata['City']=='Hyderabad, India','Amount'].quantile(0.75))
print('75th Percentile of Amount Spent by Kolkata: ', fdata.loc[fdata['City']=='Kolkata, India','Amount'].quantile(0.75))

fdata.loc[(fdata['Amount']>231531.75) & (fdata['City']=='Delhi, India'), 'MetroCity_Check']=1
fdata.loc[(fdata['Amount']>231531.75) & (fdata['City']=='Greater Mumbai, India'), 'MetroCity_Check']=1
fdata.loc[(fdata['Amount']>224771.5) & (fdata['City']=='Pune, India'), 'MetroCity_Check']=1
fdata.loc[(fdata['Amount']>212849.25) & (fdata['City']=='Hyderabad, India'), 'MetroCity_Check']=1
fdata.loc[(fdata['Amount']>216616.0) & (fdata['City']=='Kolkata, India'), 'MetroCity_Check']=1

rule5 = fdata.loc[fdata['MetroCity_Check']==1]
rule5 = rule5[["City", "Date", "Card Type", "Exp Type", "Gender", "Amount"]]
print('Data Exlpoiting Rule 5')
rule5.reset_index(drop=True)

75th Percentile of Amount Spent in Delhi:  231531.75
75th Percentile of Amount Spent by Greater Mumbai:  233161.0
75th Percentile of Amount Spent by Pune:  224771.5
75th Percentile of Amount Spent by Hyderabad:  212849.25
75th Percentile of Amount Spent by Kolkata:  216616.0
Data Exlpoiting Rule 5


,City,Date,Card Type,Exp Type,Gender,Amount
0,"Delhi, India",2014-06-22,Platinum,Bills,F,280061
1,"Greater Mumbai, India",2013-12-07,Signature,Bills,F,278036
2,"Delhi, India",2014-04-27,Signature,Bills,F,254359
3,"Greater Mumbai, India",2014-08-15,Signature,Bills,F,302834
4,"Greater Mumbai, India",2014-11-28,Platinum,Bills,F,647116
...,...,...,...,...,...,...
2328,"Pune, India",2014-07-04,Signature,Travel,M,266219
2329,"Hyderabad, India",2015-05-15,Silver,Travel,F,280335
2330,"Kolkata, India",2014-10-14,Gold,Travel,M,243249
2331,"Pune, India",2014-08-03,Signature,Travel,M,246316


In [8]:
#Rule-6
metro = ['Delhi, India', 'Greater Mumbai, India', 'Pune, India', 'Hyderabad, India', 'Kolkata, India']
print('50th Percentile of AverageAmount Spent in Non-Metro Cities: ', fdata.loc[~fdata.City.isin(metro), 'Amount'].quantile(0.50))

fdata.loc[(fdata['Amount']>153289) & (fdata['City']!='Delhi, India'), 'MetroCity_Check']=1
fdata.loc[(fdata['Amount']>153289) & (fdata['City']!='Greater Mumbai, India'), 'MetroCity_Check']=1
fdata.loc[(fdata['Amount']>153289) & (fdata['City']!='Pune, India'), 'MetroCity_Check']=1
fdata.loc[(fdata['Amount']>153289) & (fdata['City']!='Hyderabad, India'), 'MetroCity_Check']=1
fdata.loc[(fdata['Amount']>153289) & (fdata['City']!='Kolkata, India'), 'MetroCity_Check']=1

rule6 = fdata.loc[fdata['MetroCity_Check']==1]
rule6 = rule6[["City", "Date", "Card Type", "Exp Type", "Gender", "Amount"]]
print('Data Exlpoiting Rule 6')
rule6.reset_index(drop=True)

50th Percentile of AverageAmount Spent in Non-Metro Cities:  153289.0
Data Exlpoiting Rule 6


,City,Date,Card Type,Exp Type,Gender,Amount
0,"Bengaluru, India",2015-05-05,Gold,Bills,F,171574
1,"Delhi, India",2014-03-28,Silver,Bills,F,192247
2,"Delhi, India",2014-06-22,Platinum,Bills,F,280061
3,"Greater Mumbai, India",2013-12-07,Signature,Bills,F,278036
4,"Delhi, India",2014-04-27,Signature,Bills,F,254359
...,...,...,...,...,...,...
13004,"Kolkata, India",2014-02-26,Signature,Travel,F,170049
13005,"Hyderabad, India",2013-10-09,Signature,Travel,M,198903
13006,"Pune, India",2014-08-03,Signature,Travel,M,246316
13007,"Hyderabad, India",2015-01-16,Silver,Travel,M,265019


In [9]:
#If all the rules are violated, the transaction is a potential fraud.

fdatac = fdata.loc[(fdata['Limit_Check'] == 1) & (fdata['ExpenseType_Check'] == 1) & (fdata['GenExp_Check'] == 1) 
                   & (fdata['MetroCity_Check'] == 1)]
fdatac = fdatac[["City", "Date", "Card Type", "Exp Type", "Gender", "Amount"]]
print('\nPotential Fraud Data')
fdatac = fdatac.sort_values('Amount').reset_index(drop=True)
fdatac


Potential Fraud Data


,City,Date,Card Type,Exp Type,Gender,Amount
0,"Bhadrachalam, India",2014-07-13,Signature,Grocery,M,250057
1,"Bengaluru, India",2014-09-22,Signature,Entertainment,F,250103
2,"Delhi, India",2014-06-22,Signature,Food,F,250133
3,"Raisen, India",2014-04-03,Signature,Bills,F,250161
4,"Ahmedabad, India",2014-04-27,Signature,Food,F,250163
...,...,...,...,...,...,...
1251,"Delhi, India",2014-06-07,Signature,Bills,F,994537
1252,"Ahmedabad, India",2014-08-27,Platinum,Bills,F,995634
1253,"Ahmedabad, India",2015-04-17,Gold,Bills,F,996291
1254,"Delhi, India",2013-10-27,Gold,Bills,F,996754
